Starting with creating a CLI version of my program.

Starting off with creating a simple version of my program that can run with csv files. Does not take into account draft position or how many players your team can fit.

In [37]:
pip install sleeper-py
# found this resource on the internet when looking for sleeper api python on accident, decided to try it out

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# import packages
import pandas as pd
import requests
from pprint import pprint
import json

In [24]:
# constants
API = "https://api.sleeper.app/v1/players/nfl"

Starting with actual code

In [25]:
# get response from api request
response = requests.get(API)
display(response)
print(response.url)

<Response [200]>

https://api.sleeper.app/v1/players/nfl


In [43]:
def draft(player_name, player_dict, user_draft):
    '''finds player and add to list of user's players
    params: player_name (string)
            player_dict (dict)
            user_draft (list)
    returns: user_draft
    '''
    found_player = False
    for key in player_dict:
        player = player_dict[key]
        full_name = (player['first_name'] + " " + player['last_name']).lower()
        if full_name == player_name.lower():
            user_draft.append(player)
            print(f"you have drafted {player_name}")
            found_player = True

    if found_player == False:
        print(f"player {player_name} not found")
        return False
    else:
        return user_draft

In [34]:
def predicted_spots(position, league_size, team_size):
    '''def predicted_spots returns all predicted draft spots the user will draft at
    
    params: position (string): position that player's first pick is
            league_size (string): number of people in the league
            team_size (string): number of players on each team
    '''
    draft_order = []
    current_pick = int(position)
    
    for round_number in range(int(team_size)):
        if round_number % 2 == 0: # Regular order for odd rounds
            current_round = list(range(1, int(league_size) + 1))
        else: # Reverse order for even rounds (snake draft)
            current_round = list(range(int(league_size), 0, -1))
        
        # Calculate total pick number
        draft_order.append(current_pick + current_round[int(position) - 1] - 1)
        current_pick += int(league_size)
    
    return draft_order

In [35]:
# testing predicted_spots
print(predicted_spots("1", "12", "15"))

[1, 24, 25, 48, 49, 72, 73, 96, 97, 120, 121, 144, 145, 168, 169]


In [36]:
def calculate_grade(user_draft, draft_positions):
    '''
    def calculate_grade calculates draft grade by predicted points

    params: user_draft (list): list of players user has drafted
            draft_positions (list): list of positions user is drafting in
    returns: grade (string): letter grade assigned to value, prints predicted score and your score as well
    '''

    predicted_score = 0
    for round in draft_positions:
        predicted_score += round
    
    print(f"your predicted score is: {predicted_score}")

    score = 0
    for player in user_draft:
        score += player['search_rank']

    print(f"your score is: {score}")

    print("your draft grade is: ")
    if score <= predicted_score - 75:
        return "S"
    if score <= predicted_score - 50:
        return "A"
    if score <= predicted_score - 25:
        return "B"
    if score <= predicted_score:
        return "C"
    if score <= predicted_score + 25:
        return "D"
    else:
        return "F"

In [37]:
def read_players(filename):
    '''def read_players reads in players into json file

    params: filename (string): filename to read
    returns: player_dict (dictionary): dictionary with all players
    '''
    with open(filename , "r") as fo:
        player_dict = json.load(fo) 
    return player_dict

In [45]:
user_draft = []
player_dict = read_players("players.json")

league_size = input("how many people are in your league? (input int, not string)")
team_size = input("how many players are on each team? (input int, not string)")
position = input("what number pick is your first pick? (input int, not string)")

draft_positions = predicted_spots(position, league_size, team_size)

x = 0
while x < int(team_size):
    player_name = input("you are currently picking at pick " + str(draft_positions[x]) + ". please input your player's full name!")

    if draft(player_name, player_dict, user_draft) == False:
        print("try again!")
    else:
        user_draft = draft(player_name, player_dict, user_draft)
        x += 1

print(calculate_grade(user_draft, draft_positions))

you have drafted christian mccaffrey
you have drafted christian mccaffrey
player cj stroud not found
try again!
player cj stroud not found
try again!
you have drafted ceedee lamb
you have drafted ceedee lamb
you have drafted puka nacua
you have drafted puka nacua
you have drafted bo nix
you have drafted bo nix
player your mom not found
try again!
you have drafted travis kelce
you have drafted travis kelce
your predicted score is: 147
your score is: 448
your draft grade is: 
F
